In [27]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p cv2,PIL,matplotlib
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard,tflite
%watermark -p onnx,onnxruntime,tensorrt,tvm
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 95))

import sys, os, io, time, random, math
import json, base64, requests, shutil
import os.path as osp
import numpy as np

def _IMPORT(x):
    try:
        x = x.strip()
        if x.startswith('https://'):
            x = x[8:]
        if not x.endswith('.py'):
            x = x + '.py'
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        else:
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s:\n%s' % (type(x), json.dumps(attrs) if dumps else attrs
    if ret:
        return result
    print(result)

numpy 1.19.5
sklearn 0.0
pandas 1.1.5
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
torch 1.8.1+cu101
torchvision 0.9.1+cu101
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0
tflite 2.4.0
onnx 1.10.1
onnxruntime 1.8.1
tensorrt not installed
tvm not installed


In [ ]:
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'erlangai-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))



In [21]:
import tensorflow as tf
from tensorflow.core.framework import types_pb2, graph_pb2, attr_value_pb2
from google.protobuf import text_format

In [22]:
MODEL_PATH = '/data/nb_data/saved_models'

## Load Saved Models

In [23]:
model_loaded = tf.saved_model.load(MODEL_PATH)
_DIR(model_loaded)

<class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'>: ["base_model", "call_and_return_all_conditional_losses", "conv_3x3_layer", "dropout_layer", "fc_layers", "graph_debug_info", "input_projection", "input_projection2", "keras_api", "pos_encoding", "pos_encoding2", "regularization_losses", "signatures", "temporal_bn_layers", "temporal_conv_layers", "tensorflow_git_version", "tensorflow_version", "trainable_variables", "transformer_layers", "transformer_layers2", "variables", "within_period_fc_layers"]


### ConcreteFunction

In [89]:
signatures = model_loaded.signatures
_DIR(signatures), '-'*90, signatures.keys()

('tensorflow.python.saved_model.signature_serialization._SignatureMap: ["get", "items", "keys", "values"]',
 '------------------------------------------------------------------------------------------',
 KeysView(_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, input_1) at 0x7FA25BC787B8>})))

In [118]:
frozen_func = signatures['serving_default']
_DIR(frozen_func), '-'*90, frozen_func.name

('tensorflow.python.saved_model.load._WrapperFunction: ["add_gradient_functions_to_graph", "add_to_graph", "captured_inputs", "function_def", "graph", "inputs", "name", "output_dtypes", "output_shapes", "outputs", "pretty_printed_signature", "structured_input_signature", "structured_outputs", "trainable_variables", "variables"]',
 '------------------------------------------------------------------------------------------',
 b'__inference_signature_wrapper_65445')

In [119]:
len(frozen_func.inputs), len(frozen_func.outputs)

(227, 2)

In [144]:
frozen_func.inputs[0:5], '-'*60, frozen_func.outputs

([<tf.Tensor 'input_1:0' shape=(None, 64, 112, 112, 3) dtype=float32>,
  <tf.Tensor 'unknown:0' shape=() dtype=resource>,
  <tf.Tensor 'unknown_0:0' shape=() dtype=resource>,
  <tf.Tensor 'unknown_1:0' shape=() dtype=resource>,
  <tf.Tensor 'unknown_2:0' shape=() dtype=resource>],
 '------------------------------------------------------------',
 [<tf.Tensor 'Identity:0' shape=(None, 64, 32) dtype=float32>,
  <tf.Tensor 'Identity_1:0' shape=(None, 64, 1) dtype=float32>])

### FunctionDef & OpDef 

In [121]:
_DIR(frozen_func.function_def), '-'*90, _DIR(frozen_func.function_def.signature), '-'*90, _DIR(frozen_func.function_def.node_def)

('tensorflow.core.framework.function_pb2.FunctionDef: ["ArgAttrEntry", "ArgAttrs", "AttrEntry", "ByteSize", "Clear", "ClearExtension", "ClearField", "ControlRetEntry", "CopyFrom", "DESCRIPTOR", "DiscardUnknownFields", "Extensions", "FindInitializationErrors", "FromString", "HasExtension", "HasField", "IsInitialized", "ListFields", "MergeFrom", "MergeFromString", "ParseFromString", "RegisterExtension", "ResourceArgUniqueIdEntry", "RetEntry", "SerializePartialToString", "SerializeToString", "SetInParent", "UnknownFields", "WhichOneof", "arg_attr", "attr", "control_ret", "node_def", "resource_arg_unique_id", "ret", "signature"]',
 '------------------------------------------------------------------------------------------',
 'tensorflow.core.framework.op_def_pb2.OpDef: ["ArgDef", "AttrDef", "ByteSize", "Clear", "ClearExtension", "ClearField", "CopyFrom", "DESCRIPTOR", "DiscardUnknownFields", "Extensions", "FindInitializationErrors", "FromString", "HasExtension", "HasField", "IsInitialized"

In [122]:
frozen_func.function_def.attr, frozen_func.function_def.signature.name

({'_construction_context': s: "kEagerRuntime"
 },
 '__inference_signature_wrapper_65445')

### FuncGraph & GraphDef

In [136]:
_DIR(frozen_func.graph)

'tensorflow.python.framework.func_graph.FuncGraph: ["add_capture", "add_to_collection", "add_to_collections", "as_default", "as_graph_def", "as_graph_element", "building_function", "capture", "capture_by_value", "capture_call_time_value", "capture_distributed_variable", "capture_eager_tensor", "captured", "captures", "clear_captures", "clear_collection", "collections", "colocate_with", "container", "control_captures", "control_dependencies", "control_outputs", "create_op", "deferred_external_captures", "deferred_internal_captures", "device", "external_captures", "finalize", "finalized", "get_all_collection_keys", "get_collection", "get_collection_ref", "get_name_scope", "get_operation_by_name", "get_operations", "get_tensor_by_name", "gradient_override_map", "graph_def_versions", "inputs", "internal_captures", "is_control_flow_graph", "is_feedable", "is_fetchable", "mark_as_unsaveable", "name", "name_scope", "outer_graph", "output_shapes", "output_types", "outputs", "pop_capture", "pre

In [138]:
graph_def = frozen_func.graph.as_graph_def()
_DIR(graph_def)

'tensorflow.core.framework.graph_pb2.GraphDef: ["ByteSize", "Clear", "ClearExtension", "ClearField", "CopyFrom", "DESCRIPTOR", "DiscardUnknownFields", "Extensions", "FindInitializationErrors", "FromString", "HasExtension", "HasField", "IsInitialized", "ListFields", "MergeFrom", "MergeFromString", "ParseFromString", "RegisterExtension", "SerializePartialToString", "SerializeToString", "SetInParent", "UnknownFields", "WhichOneof", "library", "node", "version", "versions"]'

In [143]:
frozen_func.graph.inputs[0:5], '-'*60, frozen_func.graph.outputs

([<tf.Tensor 'input_1:0' shape=(None, 64, 112, 112, 3) dtype=float32>,
  <tf.Tensor 'unknown:0' shape=() dtype=resource>,
  <tf.Tensor 'unknown_0:0' shape=() dtype=resource>,
  <tf.Tensor 'unknown_1:0' shape=() dtype=resource>,
  <tf.Tensor 'unknown_2:0' shape=() dtype=resource>],
 '------------------------------------------------------------',
 [<tf.Tensor 'Identity:0' shape=(None, 64, 32) dtype=float32>,
  <tf.Tensor 'Identity_1:0' shape=(None, 64, 1) dtype=float32>])

In [148]:
operations = frozen_func.graph.get_operations()
_DIR(operations), '-'*60, _DIR(operations[0])

('list: ["append", "clear", "copy", "count", "extend", "index", "insert", "pop", "remove", "reverse", "sort"]',
 '------------------------------------------------------------',
 'tensorflow.python.framework.ops.Operation: ["colocation_groups", "control_inputs", "device", "get_attr", "graph", "inputs", "name", "node_def", "op_def", "outputs", "run", "traceback", "type", "values"]')

In [173]:
for i, op in enumerate(operations[:2] + operations[-2:]):
    if i == 2:
        print('*'*80)
    print('inputs:', op.inputs, 'outputs:', op.outputs, 'type:', op.type, 'values:', op.values)
    print('-'*80)
    print(op)
    print('+'*80)

inputs: () outputs: [<tf.Tensor 'input_1:0' shape=(None, 64, 112, 112, 3) dtype=float32>] type: Placeholder values: <bound method Operation.values of <tf.Operation 'input_1' type=Placeholder>>
--------------------------------------------------------------------------------
name: "input_1"
op: "Placeholder"
attr {
  key: "_user_specified_name"
  value {
    s: "input_1"
  }
}
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 64
      }
      dim {
        size: 112
      }
      dim {
        size: 112
      }
      dim {
        size: 3
      }
    }
  }
}

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
inputs: () outputs: [<tf.Tensor 'unknown:0' shape=() dtype=resource>] type: Placeholder values: <bound method Operation.values of <tf.Operation 'unknown' type=Placeholder>>
--------------------------------------------------------------------

In [175]:
for op in frozen_func.graph.get_operations():
    print(op.name)

input_1
unknown
unknown_0
unknown_1
unknown_2
unknown_3
unknown_4
unknown_5
unknown_6
unknown_7
unknown_8
unknown_9
unknown_10
unknown_11
unknown_12
unknown_13
unknown_14
unknown_15
unknown_16
unknown_17
unknown_18
unknown_19
unknown_20
unknown_21
unknown_22
unknown_23
unknown_24
unknown_25
unknown_26
unknown_27
unknown_28
unknown_29
unknown_30
unknown_31
unknown_32
unknown_33
unknown_34
unknown_35
unknown_36
unknown_37
unknown_38
unknown_39
unknown_40
unknown_41
unknown_42
unknown_43
unknown_44
unknown_45
unknown_46
unknown_47
unknown_48
unknown_49
unknown_50
unknown_51
unknown_52
unknown_53
unknown_54
unknown_55
unknown_56
unknown_57
unknown_58
unknown_59
unknown_60
unknown_61
unknown_62
unknown_63
unknown_64
unknown_65
unknown_66
unknown_67
unknown_68
unknown_69
unknown_70
unknown_71
unknown_72
unknown_73
unknown_74
unknown_75
unknown_76
unknown_77
unknown_78
unknown_79
unknown_80
unknown_81
unknown_82
unknown_83
unknown_84
unknown_85
unknown_86
unknown_87
unknown_88
unknown_89
unkn

In [ ]:
## 

In [ ]:
import tensorflow as tf
from tensorflow.core.framework import types_pb2, graph_pb2, attr_value_pb2
from tensorflow.tools.graph_transforms import TransformGraph
from google.protobuf import text_format
import numpy as np

# object detection api input and output nodes
input_name = "image_tensor"
output_names = ["detection_boxes", "detection_classes", "detection_scores", "num_detections"]
# Const should be float32 in object detection api during nms (see here: https://www.tensorflow.org/api_docs/cc/class/tensorflow/ops/non-max-suppression-v4.html)
keep_fp32_node_name = ["Postprocessor/BatchMultiClassNonMaxSuppression/MultiClassNonMaxSuppression/non_max_suppression/iou_threshold",
                       "Postprocessor/BatchMultiClassNonMaxSuppression/MultiClassNonMaxSuppression/non_max_suppression/score_threshold"]

def load_graph(model_path):
    graph = tf.Graph()
    with graph.as_default():
        graph_def = tf.GraphDef()
        if model_path.endswith("pb"):
            with open(model_path, "rb") as f:
                graph_def.ParseFromString(f.read())
        else:
            with open(model_path, "r") as pf:
                text_format.Parse(pf.read(), graph_def)
        tf.import_graph_def(graph_def, name="")
        sess = tf.Session(graph=graph)
        return sess

def rewrite_batch_norm_node_v2(node, graph_def, target_type='fp16'):
    """
    Rewrite FusedBatchNorm with FusedBatchNormV2 for reserve_space_1 and reserve_space_2 in FusedBatchNorm require float32 for 
    gradient calculation (See here: https://www.tensorflow.org/api_docs/cc/class/tensorflow/ops/fused-batch-norm)
    """
    if target_type == 'fp16':
        dtype = types_pb2.DT_HALF
    elif target_type == 'fp64':
        dtype = types_pb2.DT_DOUBLE
    else:
        dtype = types_pb2.DT_FLOAT
    new_node = graph_def.node.add()
    new_node.op = "FusedBatchNormV2"
    new_node.name = node.name
    new_node.input.extend(node.input)
    new_node.attr["U"].CopyFrom(attr_value_pb2.AttrValue(type=types_pb2.DT_FLOAT))
    for attr in list(node.attr.keys()):
        if attr == "T":
            node.attr[attr].type = dtype
        new_node.attr[attr].CopyFrom(node.attr[attr])
    print("rewrite fused_batch_norm done!")

def convert_graph_to_fp16(model_path, save_path, name, as_text=False, target_type='fp16', input_name=None, output_names=None):
    if target_type == 'fp16':
        dtype = types_pb2.DT_HALF
    elif target_type == 'fp64':
        dtype = types_pb2.DT_DOUBLE
    else:
        dtype = types_pb2.DT_FLOAT
    source_sess = load_graph(model_path)
    source_graph_def = source_sess.graph.as_graph_def()
    target_graph_def = graph_pb2.GraphDef()
    target_graph_def.versions.CopyFrom(source_graph_def.versions)
    for node in source_graph_def.node:
        # fused batch norm node
        if node.op == "FusedBatchNorm":
            rewrite_batch_norm_node_v2(node, target_graph_def, target_type=target_type)
            continue
        # replicate node
        new_node = target_graph_def.node.add()
        new_node.op = node.op
        new_node.name = node.name
        new_node.input.extend(node.input)
        attrs = list(node.attr.keys())
        # keep batch norm params node
        if ("BatchNorm" in node.name) or ('batch_normalization' in node.name):
            for attr in attrs:
                new_node.attr[attr].CopyFrom(node.attr[attr])
            continue
        # replace dtype in node attr with target dtype
        for attr in attrs:
            # keep special node in fp32
            if node.name in keep_fp32_node_name:
                new_node.attr[attr].CopyFrom(node.attr[attr])
                continue
            if node.attr[attr].type == types_pb2.DT_FLOAT:
                # modify node dtype
                new_node.attr[attr].type = dtype
            if attr == "value":
                tensor = node.attr[attr].tensor
                if tensor.dtype == types_pb2.DT_FLOAT:
                    # if float_val exists
                    if tensor.float_val:
                        float_val = tf.make_ndarray(node.attr[attr].tensor)
                        new_node.attr[attr].tensor.CopyFrom(tf.make_tensor_proto(float_val, dtype=dtype))
                        continue
                    # if tensor content exists
                    if tensor.tensor_content:
                        tensor_shape = [x.size for x in tensor.tensor_shape.dim]
                        tensor_weights = tf.make_ndarray(tensor)
                        # reshape tensor
                        tensor_weights = np.reshape(tensor_weights, tensor_shape)
                        tensor_proto = tf.make_tensor_proto(tensor_weights, dtype=dtype)
                        new_node.attr[attr].tensor.CopyFrom(tensor_proto)
                        continue
            new_node.attr[attr].CopyFrom(node.attr[attr])
    # transform graph
    if output_names:
        if not input_name:
            input_name = []
        transforms = ["strip_unused_nodes"]
        target_graph_def = TransformGraph(target_graph_def, input_name, output_names, transforms)
    # write graph_def to model
    tf.io.write_graph(target_graph_def, logdir=save_path, name=name, as_text=as_text)
    print("Converting done ...")

save_path = "test"
name = "test.pb"
as_text = False
target_type = 'fp16'
convert_graph_to_fp16(model_path, save_path, name, as_text=as_text, target_type=target_type, input_name=input_name, output_names=output_names)
# test loading
# ISSUE: loading detection model is extremely slow while loading classification model is normal
sess = load_graph(save_path+"/"+name)

## References

- [post-training-quantization-of-tensorflow-model-to-fp16][1]
- [tf.compat.v1.wrap_function][2]


[2]: https://tensorflow.google.cn/api_docs/python/tf/compat/v1/wrap_function
[1]: https://medium.com/@fanzongshaoxing/post-training-quantization-of-tensorflow-model-to-fp16-8d66b9dfa77f